# Train 데이터에 있는 토큰을 분석하는 부분이다.
get-tokens에서 토큰화를 한 파일을 가지고 하면 된다.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# mount to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import text data from ../raw-data/ directory
os.getcwd()
data_dir = os.getcwd() + '/drive/MyDrive/aiffel/dlthon/raw-data'
print(data_dir)

/content/drive/MyDrive/aiffel/dlthon/raw-data


In [ ]:
train = pd.read_csv(data_dir + '/train-tokens.csv')
train.head(5)

,idx,class,conversation,morphs,nouns
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면...,"지금,너,스스로,를,죽이,어,달라,고,애원,하,는,것,이,ㄴ가,?,아니,ㅂ니다,.,...","너,스스로,애원,거,혼자,우리,사건,선택,가족,선택권,너와,한번,겠군,이의,저,발"
1,1,협박 대화,길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...,"길동,경찰서,일,ㅂ니다,.,9,시,40,분,마트,에,폭발물,을,설치,하,ㄹ,거,이,...","길동,길동경찰서,경찰서,9,9시,시,40,40분,분,마트,폭발물,설치,거,한번,얘기..."
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없...,"너,되게,귀엽,ㄴ,거,알,ㄹ지,?,나,보다,작,은,남자,는,첨,보,았,어,.,그만하...","너,거,나,남자,첨,니들,지영,키,160,재,군대,보태,보태준거,준거,난쟁이,희망,..."
3,3,갈취 대화,어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈...,"어이,거기,예,??,느,어,말,이야,느,어,.,이리,오라,이,고,무슨,일,.,느,어...","어이,거기,예,말,오라,일,옷,얘,돈,돈,너,오늘,피시,피시방,방,콜,마지막,기회,거"
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,"저기,요,혹시,날,이,너무,뜨겁,잖아요,?,저희,회사,에서,이,선,크림,팔,는데,한...","저기,날,저희,회사,선,선크림,크림,번,손등,진짜,필요,중,여기,성분,거,저,피부,..."


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3950 non-null   int64 
 1   class         3950 non-null   object
 2   conversation  3950 non-null   object
 3   morphs        3950 non-null   object
 4   nouns         3945 non-null   object
dtypes: int64(1), object(4)
memory usage: 154.4+ KB


## 클래스 별로 토큰을 분석해보도록 하자.
다음 두 종류로 만들어진 토큰을 살펴보자.
- 형태소 별 토큰(morphs 칼럼)
- 명사 별 토큰(nouns 칼럼)

협박 대화 = 0,
갈취 대화 = 1,
직장 내 괴롭힘 대화 = 2,
기타 괴롭힘 대화 = 3,
일반 대화 = 4

In [ ]:
# 데이터셋에서 토큰을 가져오는 함수 정의
def get_tokens(dataset):
    tokens = []
    for sentence in dataset:
        for word in str(sentence).split(','):
            tokens.append(word)
    return tokens

In [ ]:
# 형태소 토큰을 클래스별로 가져온다.
morph_tokens_0 = get_tokens(train[train['class'] == '협박 대화']['morphs'])
morph_tokens_1 = get_tokens(train[train['class'] == '갈취 대화']['morphs'])
morph_tokens_2 = get_tokens(train[train['class'] == '직장 내 괴롭힘 대화']['morphs'])
morph_tokens_3 = get_tokens(train[train['class'] == '기타 괴롭힘 대화']['morphs'])

print(len(morph_tokens_0))
print(len(morph_tokens_1))
print(len(morph_tokens_2))
print(len(morph_tokens_3))

128857
123908
132713
134220


In [ ]:
# 명사 토큰을 클래스별로 가져온다.
noun_tokens_0 = get_tokens(train[train['class'] == '협박 대화']['nouns'])
noun_tokens_1 = get_tokens(train[train['class'] == '갈취 대화']['nouns'])
noun_tokens_2 = get_tokens(train[train['class'] == '직장 내 괴롭힘 대화']['nouns'])
noun_tokens_3 = get_tokens(train[train['class'] == '기타 괴롭힘 대화']['nouns'])

print(len(noun_tokens_0))
print(len(noun_tokens_1))
print(len(noun_tokens_2))
print(len(noun_tokens_3))

24340
25255
27239
26572


In [ ]:
# 토큰의 빈도수를 분석해보자.
from collections import Counter

def token_counter(tokens, top_n:int=None):
    # top_n이 None이면 모든 토큰의 빈도수를 세서 반환한다.
    # top_n이 100이면 가장 많은 100개 토큰의 빈도수를 세서 반환한다.
    counter = Counter(tokens)
    if(top_n == None):
        return counter
    else:
        return counter.most_common(top_n)

In [ ]:
# 형태소 토큰의 빈도수 보자
print("빈도수 별로 나타내는 협박 대화의 형태소 토큰")
print(token_counter(morph_tokens_0, 25))
print()
print("빈도수 별로 나타내는 갈취 대화의 형태소 토큰")
print(token_counter(morph_tokens_1, 25))
print()
print("빈도수 별로 나타내는 직장 내 괴롭힘 대화의 형태소 토큰")
print(token_counter(morph_tokens_2, 25))
print()
print("빈도수 별로 나타내는 기타 괴롭힘 대화의 형태소 토큰")
print(token_counter(morph_tokens_3, 25))
print()



빈도수 별로 나타내는 협박 대화의 형태소 토큰
[('.', 5784), ('어', 5734), ('하', 4358), ('이', 3656), ('?', 3388), ('아', 2467), ('는', 2225), ('가', 2161), ('고', 1949), ('ㄹ', 1707), ('야', 1677), ('나', 1516), ('었', 1498), ('ㄴ', 1418), ('을', 1337), ('지', 1312), ('거', 1301), ('에', 1223), ('게', 1175), ('니', 1105), ('네', 957), ('면', 948), ('너', 941), ('도', 907), ('내', 892)]

빈도수 별로 나타내는 갈취 대화의 형태소 토큰
[('.', 6214), ('어', 5171), ('이', 3949), ('?', 3688), ('아', 3529), ('하', 3148), ('가', 1797), ('고', 1747), ('는', 1591), ('야', 1552), ('돈', 1487), ('나', 1423), ('ㄴ', 1415), ('주', 1243), ('에', 1220), ('ㄹ', 1082), ('네', 1075), ('없', 1057), ('었', 1016), ('만', 987), ('거', 970), ('다', 926), ('있', 922), ('면', 912), ('게', 895)]

빈도수 별로 나타내는 직장 내 괴롭힘 대화의 형태소 토큰
[('.', 6048), ('하', 5977), ('이', 4811), ('?', 4387), ('어', 3439), ('아', 2847), ('는', 2353), ('가', 2119), ('고', 1979), ('ㄴ', 1815), ('네', 1640), ('ㅂ니다', 1433), ('에', 1377), ('었', 1372), ('습니다', 1121), ('거', 1118), ('도', 1111), ('ㄹ', 1108), ('것', 1093), ('나', 1076), ('야', 10

In [ ]:
# 명사 토큰의 빈도수 보자
NUM_WORDS = 50

print("빈도수 별로 나타내는 협박 대화의 명사 토큰")
print(token_counter(noun_tokens_0, NUM_WORDS))
print()
print("빈도수 별로 나타내는 갈취 대화의 명사 토큰")
print(token_counter(noun_tokens_1, NUM_WORDS))
print()
print("빈도수 별로 나타내는 직장 내 괴롭힘 대화의 명사 토큰")
print(token_counter(noun_tokens_2, NUM_WORDS))
print()
print("빈도수 별로 나타내는 기타 괴롭힘 대화의 명사 토큰")
print(token_counter(noun_tokens_3, NUM_WORDS))
print()



빈도수 별로 나타내는 협박 대화의 명사 토큰
[('거', 625), ('내', 521), ('나', 506), ('너', 484), ('말', 364), ('니', 325), ('저', 314), ('뭐', 265), ('야', 261), ('수', 253), ('내가', 250), ('우리', 230), ('사람', 213), ('발', 180), ('마', 176), ('일', 170), ('안', 165), ('때', 155), ('하나', 146), ('터', 145), ('지금', 143), ('어디', 139), ('생각', 138), ('새끼', 129), ('여기', 129), ('줄', 124), ('칼', 123), ('전', 123), ('무엇', 117), ('돈', 116), ('집', 114), ('데', 111), ('이것', 111), ('이', 109), ('당장', 108), ('미안', 106), ('신고', 103), ('정신', 103), ('시간', 97), ('가족', 95), ('당신', 95), ('이거', 91), ('앞', 88), ('경찰', 87), ('오늘', 85), ('한번', 83), ('번', 82), ('소리', 82), ('애', 81), ('날', 80)]

빈도수 별로 나타내는 갈취 대화의 명사 토큰
[('돈', 651), ('거', 554), ('나', 516), ('야', 466), ('내', 438), ('너', 344), ('말', 339), ('원', 311), ('저', 304), ('뭐', 265), ('내가', 242), ('이거', 237), ('만', 230), ('니', 206), ('안', 184), ('때', 177), ('여기', 170), ('우리', 170), ('이번', 157), ('수', 152), ('오늘', 148), ('일', 142), ('친구', 139), ('이것', 136), ('미안', 132), ('지금', 126), ('터', 125), ('

## 불용어 만들기
가장 많이 등장하는 단어들은 문법적인 요소들이거나 일반적인 단어들로 보인다. 그러므로 이러한 단어들로 불용어를 만들어보자.

In [ ]:
# 클래스 별 토큰을 합친다.

# 상위 25개의 형태소 토큰 합치기
combined_morphs = token_counter(morph_tokens_0, 25) + token_counter(morph_tokens_1, 25) + token_counter(morph_tokens_2, 25) + token_counter(morph_tokens_3, 25)
print("빈도수 별로 나타내는 전체 대화의 형태소 토큰")
print(combined_morphs)
print(len(combined_morphs))
print()

# 상위 50개의 형태소 토큰 합치기
combined_nouns = token_counter(noun_tokens_0, 50) + token_counter(noun_tokens_1, 50) + token_counter(noun_tokens_2, 50) + token_counter(noun_tokens_3, 50)
print("빈도수 별로 나타내는 전체 대화의 명사 토큰")
print(combined_nouns)
print(len(combined_nouns))

빈도수 별로 나타내는 전체 대화의 형태소 토큰
[('.', 5784), ('어', 5734), ('하', 4358), ('이', 3656), ('?', 3388), ('아', 2467), ('는', 2225), ('가', 2161), ('고', 1949), ('ㄹ', 1707), ('야', 1677), ('나', 1516), ('었', 1498), ('ㄴ', 1418), ('을', 1337), ('지', 1312), ('거', 1301), ('에', 1223), ('게', 1175), ('니', 1105), ('네', 957), ('면', 948), ('너', 941), ('도', 907), ('내', 892), ('.', 6214), ('어', 5171), ('이', 3949), ('?', 3688), ('아', 3529), ('하', 3148), ('가', 1797), ('고', 1747), ('는', 1591), ('야', 1552), ('돈', 1487), ('나', 1423), ('ㄴ', 1415), ('주', 1243), ('에', 1220), ('ㄹ', 1082), ('네', 1075), ('없', 1057), ('었', 1016), ('만', 987), ('거', 970), ('다', 926), ('있', 922), ('면', 912), ('게', 895), ('.', 6048), ('하', 5977), ('이', 4811), ('?', 4387), ('어', 3439), ('아', 2847), ('는', 2353), ('가', 2119), ('고', 1979), ('ㄴ', 1815), ('네', 1640), ('ㅂ니다', 1433), ('에', 1377), ('었', 1372), ('습니다', 1121), ('거', 1118), ('도', 1111), ('ㄹ', 1108), ('것', 1093), ('나', 1076), ('야', 1071), ('죄송', 1044), ('게', 1023), ('겠', 1006), ('지', 984), ('.',

In [ ]:
# 합친 토큰에서 빈도 수 부분은 제거하고 단어만 가져온다.

# (예시) [('거', 625), ('내', 521)] -> ['거', '내']

combined_morphs = [x[0] for x in combined_morphs]
combined_nouns = [x[0] for x in combined_nouns]

In [ ]:
# 중복된 토큰은 제거한다.
combined_morphs = list(set(combined_morphs))
combined_nouns = list(set(combined_nouns))

In [ ]:
print("형태소들")
print(' '.join(combined_morphs))
print()
print("명사들")
print(' '.join(combined_nouns))

형태소들
을 네 면 돈 도 것 하 지 어 거 에 게 니 고 나 . 가 내 ㄴ 죄송 ㄹ 었 ? 이 너 주 는 다 겠 ㅂ니다 있 습니다 아 만 야 없 왜

명사들
돈 수 말 그거 하 거 니 해 우리 이것 앞 건 씨 터 경찰 얘 거기 이 너 뭐 과장 필요 자네 당장 친구 이번 얼마 신고 집 일 날 저 소리 번 김대리 업무 요즘 원 시간 진짜 생각 전 하나 데 만 팀장 누 이거 가족 응 여기 정신 미안 거지 기분 내 무엇 당신 어디 내일 지금 제가 김 야 애 한번 얼굴 칼 안 저기 엄마 저번 때 라 줄 나 말씀 사람 새끼 내가 부장 대리 오늘 회사 마 발 개


In [ ]:
# 명사 토큰에서 부정적인 어감을 주는 단어는 불용어에서 제외하자.
negative = ['새끼', '칼', '씨', '거지', '기분', '미안', '신고', '정신']
combined_nouns
combined_nouns = [word for word in combined_nouns if word not in negative]
combined_nouns

['돈',
 '수',
 '말',
 '그거',
 '하',
 '거',
 '니',
 '해',
 '우리',
 '이것',
 '앞',
 '건',
 '터',
 '경찰',
 '얘',
 '거기',
 '이',
 '너',
 '뭐',
 '과장',
 '필요',
 '자네',
 '당장',
 '친구',
 '이번',
 '얼마',
 '집',
 '일',
 '날',
 '저',
 '소리',
 '번',
 '김대리',
 '업무',
 '요즘',
 '원',
 '시간',
 '진짜',
 '생각',
 '전',
 '하나',
 '데',
 '만',
 '팀장',
 '누',
 '이거',
 '가족',
 '응',
 '여기',
 '내',
 '무엇',
 '당신',
 '어디',
 '내일',
 '지금',
 '제가',
 '김',
 '야',
 '애',
 '한번',
 '얼굴',
 '안',
 '저기',
 '엄마',
 '저번',
 '때',
 '라',
 '줄',
 '나',
 '말씀',
 '사람',
 '내가',
 '부장',
 '대리',
 '오늘',
 '회사',
 '마',
 '발',
 '개']

In [ ]:
# 불용어들
morph_stopwords = combined_morphs
noun_stopwords = combined_nouns

## 토큰에서 불용어를 제외하고 보기

In [ ]:
def remove_stopwords(tokens, stopwords):
    return [word for word in tokens if word not in stopwords]

In [ ]:
# 불용어 적용하기
morph_tokens_0 = remove_stopwords(morph_tokens_0, morph_stopwords)
morph_tokens_1 = remove_stopwords(morph_tokens_1, morph_stopwords)
morph_tokens_2 = remove_stopwords(morph_tokens_2, morph_stopwords)
morph_tokens_3 = remove_stopwords(morph_tokens_3, morph_stopwords)

noun_tokens_0 = remove_stopwords(noun_tokens_0, noun_stopwords)
noun_tokens_1 = remove_stopwords(noun_tokens_1, noun_stopwords)
noun_tokens_2 = remove_stopwords(noun_tokens_2, noun_stopwords)
noun_tokens_3 = remove_stopwords(noun_tokens_3, noun_stopwords)

In [ ]:
# 적용된 결과 살펴보기.
print("협박 대화의 형태소 토큰")
print(morph_tokens_0[:100])
print()

print("협박 대화의 명사 토큰")
print(noun_tokens_0[:100])




협박 대화의 형태소 토큰
['지금', '스스로', '를', '죽이', '달라', '애원', 'ㄴ가', '아니', '죽', '혼자', '죽', '우리', '까지', '사건', '휘말리', '진짜', '죽이', '버리', '싶', '정말', '잘못하', '선택', '죽', '을래', '가족', '죽여주', 'ㄹ까', '정말', '잘못하', '에게', '선택권', '선택', '못하', 'ㄴ다면', '너와', '가족', '까지', '모조리', '죽이', '버리', '선택', '못하', '한번', '도와주', '세요', '그냥', '죽이', '버리', '어야', '겠군', '이의', '저', '의', '발', '돕', '세요', '길동', '경찰서', '일', '9', '시', '40', '분', '마트', '폭발물', '설치', '똑바로', '듣', '한번', '더', '얘기', 'ㄴ다', '장난', '전화', '걸', 'ㄹ지', '마시', '죠', '9', '시', '40', '분', '마트', '폭발물', '터지', '죽', '장난', '전화', '업무', '방해죄', '해당', '되', '판단', '은', '에게', '달리', '길동', '경찰서', '폭발', '물']

협박 대화의 명사 토큰
['스스로', '애원', '혼자', '사건', '선택', '선택권', '너와', '겠군', '이의', '길동', '길동경찰서', '경찰서', '9', '9시', '시', '40', '40분', '분', '마트', '폭발물', '설치', '얘기', '장난', '장난전화', '전화', '업무방해죄', '방해죄', '해당', '판단', '폭발', '물', '선생님', '진정', '이야기', '경고', '때문', '전화', '애가', '지우개', '면박', '댁', '따님', '따님분', '분', '확인', '꾸중', '마디', '구멍가게', '물건', '본', '하교', '각오', '마세', '곤란', '단지', '교육', '차원', '뿐', '보이', '아들', '그때', '

## 이제 불용어를 바탕으로 일반적인 문장을 만들어보자.

In [69]:
general_conversation = [
    ["돈 좀 빌려줄 수 있어?", "얼마나 필요한데?", "이번 달 월세가 부족해서 50만 원 정도.", "그거는 좀 많네.", "그래도 꼭 필요해서 부탁하는 거야.", "알았어, 이번 한 번만 도와줄게."],
    ["우리 오늘 뭐 먹을까?", "이것 좀 봐, 새로 생긴 식당 메뉴인데 어때?", "오, 맛있어 보인다. 거기 가자!", "응, 그럼 시간 맞춰서 나가자.", "몇 시에 만날까?", "7시쯤 어때?"],
    ["너 이번 프로젝트 어떻게 생각해?", "솔직히 말하면 좀 힘들 것 같아.", "나도 그래. 팀장님께 다시 얘기해볼까?", "응, 우리 의견을 제대로 전달해야 할 것 같아.", "그럼 내일 아침에 같이 가자.", "좋아, 준비해둘게."],
    ["여기서 뭐 하고 있어?", "그냥 생각 좀 하고 있었어.", "무슨 생각인데?", "앞으로 해야 할 일들이 너무 많아서 정리 중이야.", "그럼 내가 도와줄까?", "고마워, 네가 있으면 든든해."],
    ["김대리 요즘 바쁘지 않아?", "응, 업무가 계속 쌓여서 정신없어.", "팀장님이 너무 일을 많이 주시는 것 같아.", "맞아, 이번에는 진짜 힘들다.", "그래도 우리 같이 하면 괜찮을 거야.", "그래, 힘내자!"],
    ["저번에 얘기했던 거 기억나?", "어떤 얘기 말이야?", "우리 여행 계획 말이야.", "아, 그거! 언제 갈지 정했어?", "이번 주말 어때?", "좋아, 그럼 준비하자."],
    ["오늘 회사에서 무슨 일이 있었어?", "부장님이 갑자기 과장님한테 소리 지르셨어.", "왜 그런 거야?", "업무 실수 때문이라던데...", "진짜 분위기 안 좋겠다.", "응, 다들 긴장했어."],
    ["내일 시간 있어?", "왜? 무슨 일인데?", "같이 영화 보러 가려고.", "좋아! 무슨 영화 볼 건데?", "이번 개봉한 신작이야.", "오케이, 기대된다!"],
    ["엄마가 오늘 뭐라고 하셨어?", "돈 관리 좀 잘하라고 하셨어.", "그거 맞는 말이지. 요즘 많이 쓰는 것 같더라.", "응, 나도 반성 중이야.", "필요한 것만 사도록 노력해봐.", "그래, 그렇게 해볼게."],
    ["너 이번에 어디 갔다 왔어?", "집에 있었는데 왜 물어봐?", "뭐, 그냥 궁금해서 물어본 거야.", "응, 별일 없었어.", "그래, 알았어. 다음엔 같이 놀자!", "좋아, 약속!"],
    ["우리 친구들 모임 언제 할까?", "이번 주말은 어때?", "좋아, 다들 시간 될까?", "응, 내가 연락해볼게.", "고마워, 기대된다!", "나도, 빨리 보고 싶다."],
    ["저기 뭐 있는지 봤어?", "응, 별거 없던데 왜 물어봐?", "그냥 궁금해서. 뭔가 있을 줄 알았거든.", "없었어, 아쉽네.", "다음엔 다른 데 가보자!", "좋아, 그렇게 하자."],
    ["너 오늘 왜 이렇게 피곤해 보여?", "일 때문에 늦게까지 있었거든.", "진짜 힘들었겠다. 괜찮아?", "응, 조금 쉬면 괜찮아질 거야.", "필요하면 말해. 도와줄게.", "고마워, 괜찮아질 거야."],
    ["우리 가족끼리 여행 언제 갈까?", "이번 여름은 어때?", "좋아, 어디로 갈지 정했어?", "응, 제주도로 가는 게 어때?", "완전 좋아, 빨리 가고 싶다!", "나도, 기대돼!"],
    ["너 얼굴 왜 이렇게 안 좋아 보여?", "요즘 일이 너무 많아서 그래.", "진짜 힘들겠다. 쉬는 날 없어?", "없어, 계속 바빠...", "필요하면 나한테 말해. 도와줄게!", "고마워, 한번 생각해볼게."],
    ["맞아, 다들 바쁘기도 하고...", "응, 그래도 잘 해내겠지!", "그럼, 우리도 열심히 하자!", "좋아, 화이팅!"],
    ["오늘 날씨 진짜 좋다!", "맞아, 어디 나갈까?", "응, 공원 가는 건 어때?", "좋아, 간단히 도시락도 싸갈까?", "완전 좋아! 준비하자!", "그래! 기대된다!"],
    ["너 김대리랑 얘기 좀 했어?", "응, 이번 건 잘 해결됐대.", "다행이다. 많이 걱정했거든.", "맞아. 이제 다들 안심했을 거야.", "그래. 앞으로 더 조심해야겠지!", "맞아. 서로 도와주면서 하자."],
    ["우리 팀장님 요즘 기분 안 좋으신 것 같지 않아?", "맞아. 무슨 일 있으신 걸까?", "글쎄. 한번 여쭤볼까 싶기도 해.", "응. 너무 티 안 나게 조심스럽게 물어보자.", "그래. 내일 기회 봐서 얘기해보자!", "좋아. 그렇게 하자."],
    ["너 집에서 뭐 하고 있었어?", "그냥. 책 읽고 있었지 뭐.", "진짜? 무슨 책 읽었는데?", "오늘의 발견이라는 책이야.", "재밌겠다. 나도 한번 읽어볼게!", "추천할게. 꼭 읽어봐!"],
    ["이번 프로젝트 언제까지 끝내야 해?", "팀장님 말로는 다음 주까지래.", "시간이 좀 촉박하네...", "응, 그래서 이번 주말에도 출근해야 할 것 같아.", "같이 하면 조금 더 빨리 끝낼 수 있을 거야.", "그래, 고마워!"],
    ["너 어제 본 영화 어땠어?", "진짜 재밌었어! 너도 꼭 봐봐.", "그래? 어떤 장르야?", "액션인데 스토리도 좋았어.", "그럼 이번 주말에 볼까?", "응, 같이 가자!"],
    ["우리 다음 주에 회식 있대.", "진짜? 어디서 하는데?", "아직 장소는 안 정해졌대.", "그럼 이번엔 가까운 곳에서 하면 좋겠다.", "맞아, 멀리 가기엔 좀 힘들잖아.", "응, 팀장님께 말해볼게."],
    ["저번에 갔던 카페 기억나?", "응, 분위기 진짜 좋았지!", "이번에 또 가볼래?", "좋아! 거기 디저트도 맛있었잖아.", "맞아, 이번엔 다른 메뉴도 먹어보자.", "그래, 기대된다!"],
    ["너 오늘 왜 이렇게 기분 좋아 보여?", "좋은 소식이 있어서 그래!", "뭔데? 빨리 말해봐!", "이번에 승진했어!", "진짜? 축하해! 완전 잘됐다!", "고마워! 너 덕분이야."],
    ["우리 다음 달에 여행 갈까?", "좋아! 어디로 갈지 정했어?", "응, 일본으로 가는 게 어때?", "완전 좋아! 언제 출발할 거야?", "휴가 날짜 맞춰서 정하자.", "그래, 빨리 준비하자!"],
    ["요즘 운동하고 있어?", "응, 매일 아침에 조깅해.", "진짜? 대단하다. 힘들지 않아?", "처음엔 힘들었는데 이제는 괜찮아졌어.", "나도 한번 시작해볼까 싶다.", "같이 하면 더 재밌을 거야."],
    ["우리 반 친구들 다 모일 수 있을까?", "글쎄, 다들 바빠서 쉽지 않을 것 같아.", "그래도 한번 연락해보자.", "좋아, 내가 몇 명한테 연락해볼게.", "고마워! 다 같이 모이면 진짜 좋겠다.", "응, 꼭 성사되길 바라자."],
    ["너 이거 어디서 샀어?", "인터넷에서 주문했어.", "진짜 예쁘다. 가격은 얼마였어?", "생각보다 저렴했어. 2만 원 정도였나?", "완전 잘 샀다! 나도 사고 싶다.", "필요하면 링크 보내줄게."],
    ["너 오늘 점심 뭐 먹었어?", "그냥 회사 근처에서 간단히 먹었어.", "뭐 먹었는데?", "김치찌개! 맛있더라.", "나도 그거 먹고 싶다.", "다음에 같이 가자!"],
    ["이번에 새로 산 옷 어때?", "진짜 예쁘다! 잘 어울려.", "고마워! 너도 하나 사볼래?", "좋아, 어디서 샀어?", "동네 쇼핑몰에서 샀어.", "같이 가서 구경하자!"],
    ["너 이번 주말에 뭐 할 거야?", "아직 특별한 계획은 없어.", "그럼 같이 등산 갈래?", "좋아! 어디로 갈 건데?", "근처에 있는 산으로 가자.", "좋아, 준비해둘게."],
    ["우리 다음 회의 언제야?", "다음 주 화요일이야.", "몇 시에 시작해?", "오전 10시부터야.", "알겠어. 준비 잘 해야겠다.", "응, 발표 자료도 점검하자."],
    ["너 어제 왜 그렇게 바빴어?", "업무 마감일이라 정신없었어.", "그래도 다 끝냈어?", "응, 겨우 마무리했어.", "수고했어! 이제 좀 쉬어.", "고마워. 정말 힘들었어."],
    ["이번에 새로 나온 핸드폰 봤어?", "응, 디자인 진짜 예쁘더라.", "기능도 엄청 좋아 보이던데.", "맞아, 나도 사고 싶다.", "가격이 좀 비싸긴 하더라.", "그래도 한번 고민해봐야겠어."],
    ["우리 동아리 모임 언제 할까?", "이번 주 금요일 어때?", "좋아, 몇 시에 만날까?", "저녁 7시쯤이 좋을 것 같아.", "장소는 어디로 정했어?", "지난번 갔던 카페로 하자."],
    ["너 요즘 무슨 책 읽고 있어?", "소설 한 권 읽고 있어.", "재밌어? 제목이 뭐야?", "'시간의 주름'이라는 책이야.", "오, 나도 한번 읽어보고 싶다.", "추천할게. 꼭 읽어봐!"],
    ["요즘 날씨가 너무 춥지 않아?", "맞아. 옷을 더 껴입고 다녀야겠어.", "핫팩 하나 챙겨 다니면 좋아.", "좋은 생각이야. 너는 어떻게 버티고 있어?",
    	"나도 핫팩 들고 다녀. 진짜 유용해!", 	"그럼 나도 오늘 사러 가야겠다."],
    ["우리 집 근처에 새로 생긴 카페 알아?",
    	"응, SNS에서 봤어. 분위기 좋아 보이던데!",
    	"맞아. 디저트도 맛있다고 하더라.",
    	"그럼 이번 주말에 같이 가볼래?",
    	"좋아! 시간 맞춰서 나가자.",
    	"알았어. 기대된다!"],
    ["너 이번에 부장님께 칭찬받았다며?",
    	"응, 보고서를 잘 썼다고 하셨어.",
    	"대단하다! 축하해!",
    	"고마워. 너도 많이 도와줘서 가능했어.",
    	"별말을 다 해. 앞으로도 같이 잘 해보자.",
    	"그래, 우리 팀 최고야!"],
    ["이번 주말에 가족 모임 있지 않아?",
    	"응, 오랜만에 친척들 다 모일 거야.",
    	"좋겠다. 뭐 준비해야 해?",
    	"엄마가 음식 준비하신대.",
    	"그럼 너는 뭐 할 건데?",
    	"나는 그냥 도와드릴 거야."],
    ["우리 반 친구들이랑 언제 다시 만날까?",
    	"이번 달 안에는 한번 만나야지!",
    	"좋아. 장소는 어디로 할까?",
    	"지난번 갔던 식당 괜찮지 않아?",
    	"응, 거기 분위기도 좋았잖아.",
    	"그럼 내가 예약해볼게."],
    ["너 이번 시험 성적 어땠어?",
    	"생각보다 잘 나왔어!",
    	"축하해! 얼마나 나왔는데?",
    	"90점 넘겼어.",
    	"대단하다! 열심히 했구나.",
    	"고마워. 너도 잘했지?"],
    ["이번 영화제 가볼래?",
    	"좋아! 어떤 작품들이 상영돼?",
    	"독립영화 위주로 상영한대.",
    	"고전 영화도 있으면 좋겠다.",
    	"고전 영화 섹션도 있다고 들었어.",
    	"그럼 꼭 가보자!"],
    ["요즘 운동 시작했다며?",
    	"응, 헬스장 다니기 시작했어.",
    	"운동하는 거 힘들지 않아?",
    	"처음엔 힘들었는데 이제는 괜찮아졌어.",
    	"나도 같이 다녀볼까 생각 중이야.",
    	"같이 하면 더 재밌을 거야."],
    ["우리 회사 워크숍 언제였지?",
    	"다음 달 초라고 들었어.",
    	"고생 많겠네. 준비는 다 됐어?",
    	"아직 부족한 게 많아서 걱정이야.",
    	"필요하면 내가 도와줄게!",
    	"고마워. 정말 큰 도움이 될 거야."],
    ["너 이번에 새로 나온 게임 해봤어?",
    	"응, 진짜 재밌더라!",
    	"그래? 어떤 장르인데?",
    	"액션 RPG야. 그래픽도 엄청 좋아.",
    	"고민 중인데 나도 사볼까 싶다.",
    	"고민할 필요 없어. 바로 사!"],
    ["너 지금 뭐 하고 있니?", "아무것도 안 하고 있어.", "그럼 나 좀 도와줄 수 있겠니?", "어떤 걸 도와주면 되니?", "이 서류를 정리해주면 좋겠어.", "알았어. 바로 할게."],
    ["오늘 날씨가 왜 이렇게 춥지?", "그러게 말이야. 겨울이 다가오나 봐.", "내 코트 좀 빌려줄게.", "고마워. 진짜 춥다.", "따뜻하게 입고 다녀야 해.", "응, 알겠어."],
    ["저기 죄송한데, 길 좀 물어봐도 될까요?", "네, 어디로 가시려고 하시죠?", "이 근처에 있는 카페를 찾고 있어요.", "아, 저쪽으로 가시면 됩니다.", "감사합니다!", "천만에요."],
    ["왜 그렇게 기분이 안 좋아 보여?", "아무것도 아니야. 그냥 조금 피곤해서 그래.", "뭔가 고민이 있는 거 아니니?", "조금 있긴 한데 말하기는 좀 그래.", "그래도 나한테 말해도 돼.", "고마워. 나중에 얘기할게."],
    ["너 오늘 뭐 먹을 거야?", "글쎄, 아직 생각 안 해봤어.", "그럼 같이 밥 먹으러 갈래?", "좋아. 뭐 먹을까?", "나는 고기가 땡기는데 너는 어때?", "나도 좋아! 고기로 가자."],
    ["거기 뭐 하는 곳이지?", "저기는 새로 생긴 카페야.", "그래? 한 번 가볼까?", "좋아. 커피 마시고 싶었는데 잘 됐다.", "그럼 지금 바로 가자.", "응, 가자!"],
    ["내일 뭐 할 계획이 있니?", "아직 특별한 계획은 없는데 왜 물어봐?", "같이 영화 보러 갈래?", "좋아! 무슨 영화 볼 건데?", "이번에 새로 나온 액션 영화 어때?", "좋아! 기대된다."],
    ["왜 이렇게 늦었니?", "죄송합니다. 길이 많이 막혔습니다.", "다음부터는 시간을 더 잘 지켜야 해.", "네, 알겠습니다. 다시는 늦지 않겠습니다.", "그래, 이번만 넘어가겠다.", "감사합니다."],
    ["그거 정말 괜찮은 아이디어 같아!", "그래? 나도 그렇게 생각했어.", "그럼 팀장님께 바로 말씀드릴까?", "응, 지금 얘기하면 좋을 것 같아.", "알았어. 내가 얘기해볼게.", "좋아! 화이팅!"],
    ["왜 이렇게 조용하니?", "그냥 생각 좀 하고 있었어.", "무슨 생각인데? 나한테 말해봐.", "앞으로 해야 할 일들이 많아서 정리 중이야.", "내가 도와줄까?", "고마워. 네가 있으면 든든해."],
    ["오늘 점심 뭐 먹을 거야?", "글쎄, 아직 정하지 않았어.", "그럼 같이 먹으러 갈래?", "좋아. 뭐 먹고 싶니?", "나는 국밥이 땡기는데 너는 어때?", "나도 좋아! 국밥으로 하자."],
    ["왜 그렇게 바빠 보이니?", "오늘 해야 할 일이 너무 많아서 그래.", "뭐 도와줄 일이라도 있니?", "그럼 이 파일 좀 대신 정리해줄 수 있어?",
    	"알았어. 내가 정리할게.",
    	"고마워. 정말 큰 도움이 될 거야."],
    ["내일은 뭐 할 계획이야?",
    	"글쎄, 아직 특별한 계획은 없어.",
    	"그럼 같이 등산 갈래?",
    	"좋아! 어디로 갈 건데?",
    	"근처 산으로 가자.",
    	"좋아, 준비해둘게."],
    ["너 오늘 왜 이렇게 피곤해 보여?",
    	"일 때문에 늦게까지 있었거든.",
    	"진짜 힘들었겠다. 괜찮아?",
    	"응, 조금 쉬면 괜찮아질 거야.",
    	"필요하면 말해.",
    	"고마워"],
    ["너 오늘 뭐 했어?", "그냥 집에서 쉬었어.", "뭐 재미있는 일 없었어?", "응, 요즘 별로 할 게 없더라.", "그럼 주말에 같이 놀러 갈래?", "좋아! 어디 갈지 정해보자."],
    ["우리 이번 주말에 뭐 할까?", "글쎄, 아직 생각 안 해봤어.", "그럼 영화 보러 갈래?", "좋아! 무슨 영화 볼 건데?", "새로 나온 액션 영화 어때?", "완전 좋아! 기대된다."],
    ["너 요즘 무슨 생각해?", "그냥 이런저런 고민이 많아.", "무슨 고민인데? 나한테 말해봐.", "앞으로 뭘 해야 할지 잘 모르겠어.", "나도 그런 적 많았어. 같이 얘기해보자.", "고마워. 네가 있어서 든든해."],
    ["우리 다음에 어디 여행 갈까?", "이번엔 바다로 가는 게 어때?", "좋아! 바다 진짜 오랜만이다.", "그럼 날짜 정해서 준비하자.", "응, 내가 숙소 알아볼게.", "고마워! 빨리 가고 싶다."],
    ["너 오늘 왜 이렇게 기분 좋아 보여?", "좋은 일이 있었거든!", "뭔데? 빨리 말해봐!", "내가 좋아하는 가수가 이번에 새 앨범을 냈어!", "진짜? 나도 들어봐야겠다.", "완전 좋아! 꼭 들어봐."],
    ["나 급하게 돈이 좀 필요한데 빌려줄 수 있어?", "얼마나 필요한데?", "10만 원 정도면 돼.", "알았어. 언제까지 갚을 거야?", "이번 주 안에 꼭 갚을게.", "그래, 믿고 빌려줄게."],
    ["내 자전거 고장 났는데 네 자전거 좀 빌릴 수 있을까?", "그래, 괜찮아. 언제까지 필요해?", "일주일 정도면 될 것 같아.", "알았어. 조심히 써야 해.", "고마워! 꼭 조심히 쓸게.", "그래, 다녀와."],
    ["너 혹시 우산 있어? 나 깜빡하고 안 가져왔어.", "응, 하나 더 있어. 빌려줄게.", "진짜 고마워! 비가 갑자기 와서 당황했어.", "조심히 쓰고 꼭 돌려줘야 해!", "알겠어. 내일 바로 돌려줄게.", "그래, 알았어."],
    ["나 책 한 권만 빌릴 수 있을까?", "무슨 책 필요해?", "'시간의 주름'이라는 책이야.", "오, 그거 내가 읽던 건데 빌려줄게.", "고마워! 다 읽고 바로 돌려줄게.", "천천히 읽어도 돼."],
    ["너 혹시 충전기 여분 있어? 내 충전기가 고장 났어.", "응, 하나 더 있어. 잠깐만 기다려봐.", "진짜 고마워! 당장 필요했거든.", "조심히 쓰고 돌려줘야 해!", "알겠어. 고맙다!", "천만에."],
    ["너 어제 내가 말한 거 때문에 기분 나빴지?", "...조금 그랬어.", "정말 미안해. 그런 뜻으로 말한 게 아니었어.", "...알겠어. 다음부터는 조심해줘.", "응, 진짜 미안했어. 앞으로 조심할게.",
    	"괜찮아. 이제 신경 쓰지 마."],
    ["내가 약속을 깜빡했어... 정말 미안해!", "...너무 기다렸잖아!",
    	"정말 사정이 있었어. 다음엔 절대 안 그럴게.",
    	"알겠어. 이번만 용서할게.",
    	"고마워... 다음엔 내가 맛있는 거 살게.",
    	"그래, 꼭 약속 지켜!"],
    ["내가 실수로 네 물건 망가뜨렸어... 미안해!",
    	"뭐라고? 어떻게 된 거야?",
    	"정말 실수였어... 새 걸로 사줄게.",
    	"다음부턴 조심해야 해.",
    	"응, 다시는 이런 일 없도록 할게.",
    	"알았어. 이번엔 넘어갈게."],
    ["너한테 화냈던 거 정말 미안해...",
    	"괜찮아. 나도 조금 과민반응했었어.",
    	"고마워... 네가 이해해줘서 다행이야.",
    	"우리 앞으로는 싸우지 말자.",
    	"응, 그렇게 하자.",
    	"고마워!"],
    ["내가 어제 너를 못 도와줘서 미안했어...",
    	"괜찮아. 네 사정도 있었잖아.",
    	"고마워... 다음엔 꼭 도와줄게.",
    	"그래, 서로 도와가면서 하자.",
    	"응, 그렇게 하자.",
    	"고맙다!"],
    ["이거 얼마예요?",
    	"5만 원입니다.",
    	"조금 비싼 것 같은데요...",
    	"그럼 4만 5천 원으로 드릴게요.",
    	"고맙습니다! 바로 살게요.",
    	"감사합니다! 좋은 하루 되세요."],
    ["혹시 이 물건 파시나요?",
    	"네, 맞습니다. 어떤 색을 원하시나요?",
    	"검정색으로 주세요.",
    	"알겠습니다. 잠시만 기다려주세요.",
    	"고맙습니다!",
    	"천만에요."],
    ["이거 중고로 팔려고 하는데 얼마 받을 수 있을까요?",
    	"상태를 보니 3만 원 정도 드릴 수 있을 것 같아요.",
    	"생각보다 적네요...",
    	"더 높은 가격은 어렵습니다.",
    	"...알겠습니다. 팔게요.",
    	"고맙습니다!"],
    ["혹시 이 물건 사고 싶으신 분 있나요?",
    	"저요! 얼마에 파세요?",
    	"2만 원이면 됩니다.",
    	"상태는 괜찮나요?",
    	"네, 거의 새 거예요!",
    	"고맙습니다! 바로 살게요."],
    ["생일 축하해! 이거 선물이야!",
    	"고마워! 이거 뭐야?",
    	"네가 좋아하는 책이야!",
    	"진짜 고마워! 감동이다!",
    	"고맙긴~ 생일 축하한다!",
    	"고마워! 잘 읽을게."],
    ["이거 너한테 주려고 샀어!",
    	"...갑자기? 왜 줘?",
    	"그냥 너 생각나서 샀지~",
    	"고마워... 감동이다!",
    	"고맙긴~ 너도 나중에 맛있는 거 사줘!",
    	"알겠어! 꼭 살게."],
    ["너 요즘 뭐 하고 지내?", "그냥 집에서 쉬고 있어.", "뭐 재밌는 거 없어?", "응, 드라마 하나 보고 있는데 재밌더라.", "무슨 드라마야?", "‘이번’에 나온 신작이야. 추천해!"],
    ["우리 다음에 어디 갈까?", "이번엔 산으로 가는 게 어때?", "좋아! 등산도 오랜만이다.", "그럼 날짜 정해서 준비하자.", "응, 내가 간식 챙길게.", "고마워! 기대된다."],
    ["너 오늘 왜 이렇게 기분 안 좋아 보여?", "그냥 좀 피곤해서 그래.", "무슨 일 있어?", "요즘 고민이 많아서 그래.", "나한테 말해봐. 들어줄게.", "고마워. 나중에 얘기할게."],
    ["우리 저번에 갔던 곳 기억나?", "응, 분위기 진짜 좋았지!", "다시 한번 가볼래?", "좋아! 이번엔 다른 메뉴도 먹어보자.", "그래, 이번 주말에 가자.", "알았어! 기대된다."],
    ["너 이번에 새로 나온 게임 해봤어?", "응, 진짜 재밌더라!", "그래? 어떤 장르야?", "액션 RPG야. 그래픽도 엄청 좋아.", "나도 한번 해봐야겠다.", "같이 하면 더 재밌을 거야."],
    ["너 혹시 공구 세트 빌려줄 수 있어?", "응, 내가 가지고 있어. 언제 필요해?", "오늘 바로 써야 할 것 같아.", "알았어. 조심히 써!", "고마워! 꼭 돌려줄게.", "천만에."],
    ["내가 지금 노트북이 고장 났는데 네 거 잠깐 빌릴 수 있을까?", "그래, 괜찮아. 언제까지 필요해?", "한 이틀 정도면 될 것 같아.", "알았어. 조심히 써야 해!", "고마워! 꼭 조심히 쓸게.",
    	"그래, 다녀와."],
    ["혹시 네가 쓰던 카메라 좀 빌릴 수 있을까?",
    	"응, 내가 잘 쓰던 건데 빌려줄게.",
    	"진짜 고마워! 여행 갈 때 쓰려고 했어.",
    	"조심히 쓰고 꼭 돌려줘야 해!",
    	"알겠어. 깨끗하게 쓸게.",
    	"고마워."],
    ["너 혹시 기타 빌릴 수 있어? 나 연습 좀 하려고 해.",
    	"응, 내가 안 쓰던 거 하나 있어.",
    	"진짜? 고맙다!",
    	"연습 잘하고 꼭 돌려줘야 해.",
    	"알겠어. 연습 끝나면 바로 줄게.",
    	"고마워."],
    ["내가 어제 너한테 너무 심하게 말했지... 미안해.",
    	"...조금 서운했어.",
    	"정말 미안해. 그런 뜻은 아니었어.",
    	"알겠어. 다음부터는 조심해줘.",
    	"응, 앞으로 더 신경 쓸게.",
    	"고마워."],
    ["내가 약속을 깜빡했어... 정말 미안해!",
    	"...너무 기다렸잖아!",
    	"정말 사정이 있었어. 다음엔 절대 안 그럴게.",
    	"알겠어. 이번만 용서할게.",
    	"고마워... 다음엔 내가 맛있는 거 살게.",
    	"그래, 꼭 약속 지켜!"],
    ["내가 실수로 네 물건 망가뜨렸어... 미안해!",
    	"뭐라고? 어떻게 된 거야?",
    	"정말 실수였어... 새 걸로 사줄게.",
    	"다음부턴 조심해야 해.",
    	"응, 다시는 이런 일 없도록 할게.",
    	"알았어. 이번엔 넘어갈게."],
    ["너한테 화냈던 거 정말 미안해...",
    	"괜찮아. 나도 조금 과민반응했었어.",
    	"고마워... 네가 이해해줘서 다행이야.",
    	"우리 앞으로는 싸우지 말자.",
    	"응, 그렇게 하자.",
    	"고마워!"],
    ["이거 얼마예요?",
    	"5만 원입니다.",
    	"조금 비싼 것 같은데요...",
    	"그럼 4만 5천 원으로 드릴게요.",
    	"고맙습니다! 바로 살게요.",
    	"감사합니다! 좋은 하루 되세요."],
    ["혹시 이 물건 파시나요?",
    	"네, 맞습니다. 어떤 색을 원하시나요?",
    	"검정색으로 주세요.",
    	"알겠습니다. 잠시만 기다려주세요.",
    	"고맙습니다!",
    	"천만에요."],
    ["이거 중고로 팔려고 하는데 얼마 받을 수 있을까요?",
    	"상태를 보니 3만 원 정도 드릴 수 있을 것 같아요.",
    	"생각보다 적네요...",
    	"더 높은 가격은 어렵습니다.",
    	"...알겠습니다. 팔게요.",
    	"고맙습니다!"],
    ["혹시 이 물건 사고 싶으신 분 있나요?",
    	"저요! 얼마에 파세요?",
    	"2만 원이면 됩니다.",
    	"상태는 괜찮나요?",
    	"네, 거의 새 거예요!",
    	"고맙습니다! 바로 살게요."],
    ["생일 축하해! 이거 선물이야!",
    	"고마워! 이거 뭐야?",
    	"네가 좋아하는 책이야!",
    	"고맙다~ 감동이다!",
        ],
    ["너 오늘 얼굴이 좋아 보인다!", "정말? 어제 푹 잤거든.", "그래서 그런가 봐. 피부도 좋아 보여.", "고마워! 너도 요즘 얼굴에 생기가 돌더라.", "진짜? 다행이다.", "응, 건강해 보이니까 좋아."],
    ["너 왜 이렇게 얼굴이 창백해?", "어제 잠을 못 잤어.", "무슨 일 있었어?", "그냥 일이 많아서 그래.", "그럼 오늘은 일찍 자야겠다.", "응, 그렇게 해야겠어."],
    ["너 얼굴에 뭐가 났네?", "응, 요즘 스트레스를 많이 받아서 그런가 봐.", "피부과 한번 가보는 게 어때?", "그래야겠어. 추천해줄 곳 있어?", "응, 내가 다니는 곳 알려줄게.", "고마워!"],
    ["너 오늘 화장 다르게 했어?", "응, 새로운 스타일로 해봤어.", "진짜 잘 어울린다!", "고마워. 너도 한번 해볼래?", "좋아! 나중에 같이 해보자.", "그래, 재밌겠다!"],
    ["너 웃을 때 진짜 예쁘다!", "정말? 고마워!", "웃으니까 분위기가 확 밝아지네.", "너도 웃는 게 더 잘 어울려!", "그래? 앞으로 더 자주 웃어야겠다.", "응! 웃는 게 최고야!"],

    # 2. 외모
    ["너 요즘 살 빠졌지?", "응, 다이어트 중이야.", "진짜 효과 있나 보다!", "고마워. 아직 갈 길이 멀어.", "충분히 잘하고 있어. 응원할게!", "고마워!"],
    ["너 머리 스타일 바꿨네?", "응, 짧게 잘랐어. 어때 보여?", "완전 잘 어울려! 시원해 보인다.", "고마워. 너도 한번 짧게 잘라볼래?", "나도 고민 중이야.", "한번 해봐! 잘 어울릴 거야."],
    ["너 피부 진짜 좋다!", "정말? 요즘 관리 좀 하고 있어.", "뭘 쓰는지 알려줘!", "간단한 팩만 하고 있어.", "그거 나도 한번 써봐야겠다.", "추천할게!"],
    ["요즘 운동하니? 몸매가 좋아졌네!", "응, 헬스장 다니기 시작했어.", "효과가 바로 보이는 것 같아!", "고마워. 꾸준히 하려고 해.", "나도 같이 다닐까 고민 중이야.",
    	"같이 하면 더 재밌을 거야."],
    ["너 눈썹 정리했어? 진짜 깔끔하다!",
    	"응, 친구가 해줬어.",
    	"진짜 자연스럽게 잘 됐다.",
    	"고마워! 너도 해볼래?",
    	"좋아! 나중에 알려줘.",
    	"알겠어."],

    # 3. 옷차림
    ["오늘 옷 진짜 잘 입었다!",
    	"정말? 고마워!",
    	"색 조합이 완전 딱이야.",
    	"고민 좀 했는데 다행이다.",
    	"다음엔 나도 코디 좀 도와줘.",
    	"좋아! 언제든 말해."],
    ["너 오늘 신발 새로 샀지?",
    	"응, 어때 보여?",
    	"완전 멋있다! 어디서 샀어?",
    	"온라인에서 샀어.",
    	"나도 사고 싶다. 링크 좀 보내줘!",
    	"알겠어. 바로 보낼게."],
    ["옷 어디서 산 거야? 진짜 예쁘다!",
    	"고마워! 동네 쇼핑몰에서 샀어.",
    	"가격은 얼마였어?",
    	"생각보다 저렴했어. 3만 원 정도?",
    	"완전 잘 샀다!",
    	"고맙다~"],
    ["너 오늘 패션 완전 힙하다!",
    	"고마워! 새로운 스타일 시도해봤어.",
    	"진짜 잘 어울린다.",
    	"고맙다~ 너도 한번 입어봐!",
    	"나랑은 안 어울릴 것 같아.",
    	"그런 말 하지 마~ 도전해봐!"],
    ["오늘 모자 쓰니까 더 멋있다!",
    	"고마워! 모자는 필수템이지.",
    	"진짜 잘 어울려!",
    	"고맙다~ 너도 하나 사서 써봐.",
    	"좋아! 어디서 샀는지 알려줘.",
    	"알겠어. 바로 알려줄게."],
    ["너 키 더 큰 것 같아!",
    	"정말? 그런 말 많이 들어.",
    	"운동해서 그런가 봐.",
    	"고마워. 꾸준히 하려고 해.",
    	"나도 키 크고 싶다...",
    	"운동하면 도움 될 거야!"],
    ["키 큰 사람들은 옷 입기가 편하겠다...",
    	"꼭 그런 건 아니야.",
    	"그래도 비율이 좋아 보이잖아!",
    	"고맙다~ 너도 충분히 멋있어.",
    	"고마워... 위로된다!",
    	"진심이야."],
    ["키 작은 게 컴플렉스인데...",
    	"그런 말 하지 마. 매력 있어!",
    	"...정말 그렇게 생각해?",
    	"당연하지. 키가 전부는 아니잖아.",
    	"고마워... 힘난다!",
    	"힘내자!"],
    ["키 크면 좋은 점 뭐가 있을까?",
    	"...글쎄, 높은 곳에 있는 물건을 쉽게 꺼낼 수 있지 않을까?",
    	"그거 말고는 별로 없는 것 같아...",
    	"고민할 필요 없어. 너 그대로 멋있으니까!",
    	"고맙다~",
    	"고마워할 필요 없어."],
    ["너 키가 진짜 크다! 몇이야?", "나 185cm야.", "와, 진짜 부럽다!", "고마워. 근데 키 크면 불편한 점도 많아.", "예를 들면 뭐가 있어?", "대중교통에서 머리가 닿을 때가 많아."],
    ["키 작은 사람도 매력 있지 않아?", "그럼! 키는 중요하지 않아.", "진짜 그렇게 생각해?", "응, 중요한 건 자신감이야.", "고마워. 너 덕분에 힘난다!", "그래, 자신감 있게 살아!"],
    ["너 키가 더 큰 것 같아!", "그래? 나도 그런 것 같아.", "운동해서 그런가 봐.", "응, 요즘 스트레칭을 많이 했거든.", "나도 해야겠다. 알려줘!", "좋아. 같이 해보자!"],
    ["키 크면 운동하기도 좋지 않아?", "응, 농구 같은 건 유리하지.", "맞아. 너 농구 잘하잖아!", "고마워. 너도 잘하잖아.", "나는 그냥 즐기는 정도야.", "그게 제일 중요한 거지!"],
    ["키 작은 게 컴플렉스인데...", "그런 말 하지 마. 매력 있어!", "...정말 그렇게 생각해?", "당연하지. 키가 전부는 아니잖아.", "고마워... 힘난다!", "힘내자!"],

    # 5. 융통성
    ["너는 진짜 융통성이 좋은 것 같아.", "정말? 고마워!", "어떻게 그렇게 잘 대처해?", "그냥 상황에 맞게 생각하려고 해.", "나도 너처럼 되고 싶다.", "연습하면 될 거야!"],
    ["융통성 없는 사람이랑 일하기 힘들지 않아?", "맞아, 가끔 답답할 때가 있어.", "그래도 참을 수밖에 없잖아.", "응, 최대한 이해하려고 노력해.", "너 진짜 대단하다!",
    	"고마워!"],
    ["너는 융통성이 부족하다는 말 들어본 적 있어?",
    	"응, 가끔 들어.",
    	"그럼 어떻게 바꾸려고 노력해?",
    	"상대방 입장에서 생각하려고 해.",
    	"그거 좋은 방법이다!",
    	"고마워."],
    ["융통성이란 게 뭘까?",
    	"상황에 따라 유연하게 대처하는 거 아닐까?",
    	"맞아. 나도 더 융통성 있게 살고 싶어.",
    	"충분히 잘하고 있어.",
    	"고마워. 더 노력할게.",
    	"응원할게!"],
    ["너는 융통성이 부족하다는 말 들어본 적 있어?",
    	"응, 가끔 들어.",
    	"그럼 어떻게 바꾸려고 노력해?",
    	"상대방 입장에서 생각하려고 해.",
    	"그거 좋은 방법이다!",
    	"고마워."],

    # 6. 능력
    ["너는 진짜 능력이 뛰어난 것 같아!",
    	"정말? 고마워!",
    	"어떻게 그렇게 잘해?",
    	"그냥 꾸준히 연습했어.",
    	"나도 너처럼 되고 싶다.",
    	"고맙다~ 같이 열심히 하자."],
    ["능력 있는 사람이 되고 싶어...",
    	"너도 충분히 능력 있어.",
    	"...정말 그렇게 생각해?",
    	"당연하지. 네가 얼마나 열심히 하는데!",
    	"고마워. 더 열심히 해야겠다.",
    	"응원할게!"],
    ["능력을 키우려면 뭘 해야 할까?",
    	"꾸준히 노력하고 배우는 게 중요하지 않을까?",
    	"맞아. 나도 그렇게 생각해.",
    	"너라면 충분히 잘할 수 있을 거야.",
    	"고마워! 힘내볼게.",
    	"화이팅!"],
    ["너는 능력이 뛰어나서 부럽다...",
    	"고맙다~ 하지만 나도 부족한 점이 많아.",
    	"그래도 넌 항상 잘하잖아!",
    	"고맙다~ 너도 충분히 잘하고 있어.",
    	"고마워... 힘난다!",
    	"같이 열심히 하자."],
    ["능력이란 게 뭘까?",
    	"자신의 일을 잘 해내는 거 아닐까?",
    	"맞아. 나도 그런 사람이 되고 싶어.",
    	"충분히 가능할 거야!",
    	"고마워. 더 노력해야겠다.",
    	"응원할게!"],

    # 7. 돈
    ["돈은 정말 중요한 것 같아.",
    	"맞아, 없으면 불편한 게 많지.",
    	"그래서 돈을 모으려고 노력 중이야.",
    	"좋은 생각이다! 나도 시작해야겠다.",
    	"같이 하면 더 좋을 것 같아!",
    	"고맙다~ 같이 해보자."]
]


In [89]:
# general conversation 전처리하기
concat_conv = []
for conversation in general_conversation:
    dl = ""
    for sentence in conversation:
        dl += sentence
        dl += " "
    dl = dl.replace(',', '')
    concat_conv.append([dl])
concat_conv[:10]

[['돈 좀 빌려줄 수 있어? 얼마나 필요한데? 이번 달 월세가 부족해서 50만 원 정도. 그거는 좀 많네. 그래도 꼭 필요해서 부탁하는 거야. 알았어 이번 한 번만 도와줄게. '],
 ['우리 오늘 뭐 먹을까? 이것 좀 봐 새로 생긴 식당 메뉴인데 어때? 오 맛있어 보인다. 거기 가자! 응 그럼 시간 맞춰서 나가자. 몇 시에 만날까? 7시쯤 어때? '],
 ['너 이번 프로젝트 어떻게 생각해? 솔직히 말하면 좀 힘들 것 같아. 나도 그래. 팀장님께 다시 얘기해볼까? 응 우리 의견을 제대로 전달해야 할 것 같아. 그럼 내일 아침에 같이 가자. 좋아 준비해둘게. '],
 ['여기서 뭐 하고 있어? 그냥 생각 좀 하고 있었어. 무슨 생각인데? 앞으로 해야 할 일들이 너무 많아서 정리 중이야. 그럼 내가 도와줄까? 고마워 네가 있으면 든든해. '],
 ['김대리 요즘 바쁘지 않아? 응 업무가 계속 쌓여서 정신없어. 팀장님이 너무 일을 많이 주시는 것 같아. 맞아 이번에는 진짜 힘들다. 그래도 우리 같이 하면 괜찮을 거야. 그래 힘내자! '],
 ['저번에 얘기했던 거 기억나? 어떤 얘기 말이야? 우리 여행 계획 말이야. 아 그거! 언제 갈지 정했어? 이번 주말 어때? 좋아 그럼 준비하자. '],
 ['오늘 회사에서 무슨 일이 있었어? 부장님이 갑자기 과장님한테 소리 지르셨어. 왜 그런 거야? 업무 실수 때문이라던데... 진짜 분위기 안 좋겠다. 응 다들 긴장했어. '],
 ['내일 시간 있어? 왜? 무슨 일인데? 같이 영화 보러 가려고. 좋아! 무슨 영화 볼 건데? 이번 개봉한 신작이야. 오케이 기대된다! '],
 ['엄마가 오늘 뭐라고 하셨어? 돈 관리 좀 잘하라고 하셨어. 그거 맞는 말이지. 요즘 많이 쓰는 것 같더라. 응 나도 반성 중이야. 필요한 것만 사도록 노력해봐. 그래 그렇게 해볼게. '],
 ['너 이번에 어디 갔다 왔어? 집에 있었는데 왜 물어봐? 뭐 그냥 궁금해서 물어본 거야. 응 별일 없었어. 그래 알았어. 다음엔 같이 놀자! 좋아 약속! ']]

In [95]:
# 원본 train data 불러오기
df = pd.read_csv(data_dir + '/train.csv')
df.drop(columns=['idx'], inplace=True, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class         3950 non-null   object
 1   conversation  3950 non-null   object
dtypes: object(2)
memory usage: 61.8+ KB


In [91]:
conv_df = pd.DataFrame(concat_conv, columns=['conversation'])
conv_df['class'] = '일반 대화'

In [98]:
merged = pd.concat([df, conv_df])
merged

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
130,일반 대화,능력 있는 사람이 되고 싶어... 너도 충분히 능력 있어. ...정말 그렇게 생각해...
131,일반 대화,능력을 키우려면 뭘 해야 할까? 꾸준히 노력하고 배우는 게 중요하지 않을까? 맞아....
132,일반 대화,너는 능력이 뛰어나서 부럽다... 고맙다~ 하지만 나도 부족한 점이 많아. 그래도 ...
133,일반 대화,능력이란 게 뭘까? 자신의 일을 잘 해내는 거 아닐까? 맞아. 나도 그런 사람이 되...


In [102]:
# add idx column
merged['idx'] = range(len(merged))
merged

,class,conversation,idx
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,1
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,2
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,3
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,4
...,...,...,...
130,일반 대화,능력 있는 사람이 되고 싶어... 너도 충분히 능력 있어. ...정말 그렇게 생각해...,4080
131,일반 대화,능력을 키우려면 뭘 해야 할까? 꾸준히 노력하고 배우는 게 중요하지 않을까? 맞아....,4081
132,일반 대화,너는 능력이 뛰어나서 부럽다... 고맙다~ 하지만 나도 부족한 점이 많아. 그래도 ...,4082
133,일반 대화,능력이란 게 뭘까? 자신의 일을 잘 해내는 거 아닐까? 맞아. 나도 그런 사람이 되...,4083


In [103]:
merged = merged[['idx', "class", "conversation"]]
merged

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
130,4080,일반 대화,능력 있는 사람이 되고 싶어... 너도 충분히 능력 있어. ...정말 그렇게 생각해...
131,4081,일반 대화,능력을 키우려면 뭘 해야 할까? 꾸준히 노력하고 배우는 게 중요하지 않을까? 맞아....
132,4082,일반 대화,너는 능력이 뛰어나서 부럽다... 고맙다~ 하지만 나도 부족한 점이 많아. 그래도 ...
133,4083,일반 대화,능력이란 게 뭘까? 자신의 일을 잘 해내는 거 아닐까? 맞아. 나도 그런 사람이 되...


In [104]:
merged.to_csv(data_dir + '/train_with_gen_conv.csv', index=False)